In [28]:
import pandas as pd
import dwsemear

import postgresemear as sql

dwsemear.atualize_metadados()

In [2]:
import datetime
def converte_data_forte(formato):
    ''' Conversao de data na forma funcional. 
        Quando voce adicionar o formato,
        ele ira retornar uma funcao que realiza a conversao da data aplicando
        aquele formato'''
    def converte_data(data):
        try:
            datahora = datetime.datetime.strptime(data, formato)
            return datahora
        except:
            return None
    
    return converte_data


# TRANSFORMACOES

In [35]:
#crivo = pd.read_csv('DadosCrivo-NOV2018.csv', sep=';', encoding='latin-1',dtype=str)
crivo = pd.read_csv('190903_crivo.csv',sep=';', encoding='latin-1',dtype=str)

In [36]:
crivo = dwsemear.limpa_nome_das_colunas(crivo)
crivo.loc[:,'CPF'] = crivo.loc[:,'CPF'].apply(dwsemear.cpf_para_inteiro)
crivo = dwsemear.crivo_csv.geracao_coluna_datahora(crivo)
coluna = 'Data Nascimento Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(converte_data_forte('%d/%m/%Y'))
crivo['idade'] = dwsemear.calcula_idade(crivo, 'datahora', coluna) # idade
coluna = 'Data Admiss o Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(converte_data_forte('%d/%m/%Y'))
crivo['admissao_meses'] = dwsemear.calcula_idade(crivo, 'datahora', coluna, 30) # tempo_emprego
coluna = 'Data Resid ncia Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(converte_data_forte('%d/%m/%Y'))
crivo['residencia_meses'] = dwsemear.calcula_idade(crivo, 'datahora', coluna, 30) # tempo_moradia

In [37]:
coluna = 'Valor Renda Banco Semear Consulta Proposta JRetail PF '
crivo[coluna] = crivo[coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Renda presumida Unitfour Consulta perfil PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Discreta CEP Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(dwsemear.converte_latin_para_float)
coluna = 'Discreta J cliente Banco Semear Consulta Proposta JRetail PF '
crivo.loc[:,coluna] = crivo.loc[:,coluna].apply(lambda x: 1 if x == 'N' else 0)

#coluna = 'Discreta Categoria profissional Banco Semear Consulta Proposta JRetail PF '
#crivo.loc[:,coluna]  -- profissao -> texto
#C digo da filial do lojista  texto
#C digo do lojista

In [38]:
crivo = dwsemear.crivo_csv.geracao_coluna_ultimo_pedido(crivo) # passo lento

# OBTENCAO DO SCORE

In [39]:
coluna_score1 = 'Valor Score 12 meses Banco Semear SPC MIX SPC CHEQUE PF ' # score 0-100
crivo.loc[:,coluna_score1] = crivo.loc[:,coluna_score1].apply(dwsemear.converte_latin_para_float)
coluna_score2 = 'Valor - Score 12 meses(SPC BrasilV2 - SPC MIX (SPC + CHEQUE) .. - PF)' # score 0-1000
score = []
if coluna_score2 in crivo.columns:
    crivo.loc[:,coluna_score2] = crivo.loc[:,coluna_score2].apply(dwsemear.converte_latin_para_float)
    for i in crivo.index:
        if dwsemear.e_nulo(crivo.loc[i,coluna_score2]):
            score.append(crivo.loc[i,coluna_score1] * 10)
        else:
            score.append(crivo.loc[i,coluna_score2])
    crivo['score'] = score
else:
    crivo['score'] = crivo.loc[:,coluna_score1] * 10


# CONFERENCIA DE NOT NULL E SE OS REGISTROS JA FORAM ADICIONADOS

In [40]:
metadados = dwsemear.pegue_metadados('originacao')

In [41]:
crivo.loc[:,'Codigo proposta'].isnull().sum()

0

In [42]:
crivo.loc[:,'CPF'].isnull().sum()

0

In [43]:
crivo.loc[:,'datahora'].isnull().sum()

0

# DELETANDO PEDIDOS QUE JA FORAM ADICIONADOS

In [48]:
nome_tabela = 'originacao'
nome_coluna = 'datahora_pedido'
query_max = "SELECT MAX(" + nome_coluna + ") FROM " + nome_tabela
sql.query_para_pandas(query_max)

,0
0,2018-11-05 14:39:51


In [50]:
ultimo_realizado = sql.query_para_pandas(query_max).iloc[0,0]

In [56]:
crivo_back = crivo.copy()

In [57]:
crivo = crivo[crivo.datahora > ultimo_realizado]  # descartando o ultimo

In [ ]:
dwsemear.metadados.leia_metadados()

# CARGA

In [59]:
colunas_map = {}
colunas_map['associados_distintos_30_dias'] = 'Quantidade Total de associados distintos nos ltimos 30 dias nas consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
colunas_map['atraso_medio'] = 'Valor Atraso m dio Banco Semear Consulta Proposta JRetail PF '
colunas_map['categoria_profissional'] = 'Discreta Categoria profissional Banco Semear Consulta Proposta JRetail PF '
colunas_map['cep'] = 'Discreta CEP Banco Semear Consulta Proposta JRetail PF '
colunas_map['cliente_novo'] = 'Discreta J cliente Banco Semear Consulta Proposta JRetail PF '
colunas_map['codigo_lojista'] = 'C digo do lojista'
colunas_map['codigo_lojista_filial'] = 'C digo da filial do lojista'
colunas_map['codigo_proposta'] = 'Codigo proposta'
colunas_map['cpf'] = 'CPF'
colunas_map['datahora_pedido'] = 'datahora'
colunas_map['genero'] = 'Discreta Sexo Banco Semear Consulta Proposta JRetail PF '
colunas_map['idade'] = 'idade'
colunas_map['quantidade_consultas'] = 'Quantidade Total de consultas realizadas Banco Semear SPC MIX SPC CHEQUE PF '
colunas_map['quantidade_parcelas_plano'] = 'Quantidade Parcelas da compra Banco Semear Consulta Proposta JRetail PF '
colunas_map['renda_cliente'] = 'Valor Renda Banco Semear Consulta Proposta JRetail PF '
colunas_map['renda_presumida_unitfour'] = 'Valor Renda presumida Unitfour Consulta perfil PF '
colunas_map['score'] = 'score'
colunas_map['tempo_emprego'] = 'admissao_meses'
colunas_map['tempo_moradia'] = 'residencia_meses'
colunas_map['uf'] = 'Discreta UF Banco Semear Consulta Proposta JRetail PF '
colunas_map['ultimo_pedido'] = 'ultimo_pedido'
colunas_map['valor_parcela'] = 'Valor Parcela da compra Banco Semear Consulta Proposta JRetail PF '

In [65]:
id_originacao = sql.query_max_coluna('id', 'originacao')
if id_originacao == None:
    id_originacao = -1
print(id_originacao)

254491


In [61]:
registros = []
for i in crivo.index:
    id_originacao += 1
    registros.append((
        dwsemear.para_int(crivo.loc[i,colunas_map['associados_distintos_30_dias']]),
        dwsemear.para_float(crivo.loc[i,colunas_map['atraso_medio']]),
        dwsemear.para_str(crivo.loc[i,colunas_map['categoria_profissional']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['cep']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['cliente_novo']]),
        dwsemear.para_str(crivo.loc[i,colunas_map['codigo_lojista']]),
        dwsemear.para_str(crivo.loc[i,colunas_map['codigo_lojista_filial']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['codigo_proposta']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['cpf']]),
        dwsemear.para_data(crivo.loc[i,colunas_map['datahora_pedido']]),
        dwsemear.para_str(crivo.loc[i,colunas_map['genero']]),
        id_originacao,
        dwsemear.para_int(crivo.loc[i,colunas_map['idade']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['quantidade_consultas']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['quantidade_parcelas_plano']]),
        dwsemear.para_float(crivo.loc[i,colunas_map['renda_cliente']]),
        dwsemear.para_float(crivo.loc[i,colunas_map['renda_presumida_unitfour']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['score']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['tempo_emprego']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['tempo_moradia']]),
        dwsemear.para_str(crivo.loc[i,colunas_map['uf']]),
        dwsemear.para_int(crivo.loc[i,colunas_map['ultimo_pedido']]),
        dwsemear.para_float(crivo.loc[i,colunas_map['valor_parcela']])
    ))


In [64]:
dwsemear.insira_tabela('originacao', registros)

In [29]:
#dwsemear.delete_tabela('originacao')
#dwsemear.crie_tabela('originacao')

In [ ]:
for col in crivo.columns:
    if 'liente' in col:
        print(col)